# Влияние ошибок округления на решение СЛАУ. Числа обусловленности.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from scipy.linalg import hilbert

In [ ]:
m1 = np.matrix('-400.60 199.80; 1198.80 -600.40')  # Первый вариант матрицы из методички Пакулиной

m2 = np.matrix(hilbert(2))  # Матрица Гильберта второго порядка

m3 = np.matrix(hilbert(5))  # Матрица Гильберта пятого порядка

In [ ]:
def norm(matrix):  # Спектральная норма матрицы
    return np.linalg.norm(x=matrix, ord=2)


def det(matrix):  # Определитель матрицы
    return np.linalg.det(matrix)

In [ ]:
def cond1(matrix):  # Спектральный критерий обусловленности
    return norm(matrix) * norm(matrix.getI())


def cond2(matrix):  # Критерий Ортеги
    res = 1
    for n in range(matrix.shape[0]):
        sum_ = 0
        for m in range(matrix.shape[0]):
            sum_ += matrix[n, m]**2
        res *= np.sqrt(sum_)
    return res/det(matrix)


def cond3(matrix):  # Угловой критерий
    res = 0
    inv_matrix = matrix.getI()
    for i in range(matrix.shape[0]):
        a = norm(matrix[i, :]) * norm(inv_matrix[:, i])
        if a > res:
            res = a
    return res

In [ ]:
def randomx(n):  # Генерирует вектор-столбец x из отрезка [0, 1] 
    return(np.matrix(np.random.rand(n, 1)))


def constx(n):  # Возвращает вектор-столбец из единиц
    return np.matrix(("1;"*n)[:-1])

In [ ]:
def computation(m, x):
    def variation(n):
        nonlocal m, b, x, conds, errors
        varm = m + n
        varb = b + n
        varx = varm.getI() * varb
        conds.append((cond1(varm), cond2(varm), cond3(varm)))
        errors.append((norm(varx - x)))
        print(f'n = {n}\nСпектральный критерий: {conds[-1][0]}\nКритерий Ортеги: {conds[-1][1]}\nУгловой критерий: {conds[-1][2]}\nМодуль невязки x: {errors[-1]}')
    b = m * x
    conds = []
    errors = []
    print("Ax = b")
    print("A =\n", m)
    print("x =\n", x)
    print("b =\n", b)
    variation(0)
    for i in range(-10, -1, 2):  # Варьируем матрицы
        variation(10**i)
    for j in range(3):  # Выводим график зависиости погрешности от числа обусловленности
        sn.scatterplot(x=([i[j] for i in conds]), y=(errors))
        plt.xlabel(f"Число обусловленности {j+1}")
        plt.ylabel("Погрешность решения x")
        plt.show()
        print(f"Зависимость между величиной {j+1}-го числа обусловленности и погрешностью решения(коэффициент пирсона):",
              np.corrcoef([i[j] for i in conds], errors)[0][1])

In [ ]:
test = np.matrix(np.random.rand(5, 5))  
# проверка зависимости погрешности решения от 
# чисел обусловленности на случайных матрицах
testx = randomx(5)

computation(test, testx)

In [ ]:
computation(m1, randomx(2))

In [ ]:
computation(m1, constx(2))

In [ ]:
computation(m2, randomx(2))

In [ ]:
computation(m2, constx(2))

In [ ]:
computation(m3, randomx(5))

In [ ]:
computation(m3, constx(5))

In [ ]:
x = [1, 2, 0]
y = [1, 2, 10000]
np.corrcoef(x, y)